In [13]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
pd.options.display.float_format = '{:,.3f}'.format
import argparse
pd.options.mode.chained_assignment = None

import pickle as pkl

from scipy.stats.stats import pearsonr
from scipy.stats.stats import pearsonr

#br_to_us=pd.read_excel("../data/Book.xlsx",skiprows=[0])
#br_to_us_dict=dict(zip(br_to_us.UK.tolist(),br_to_us.US.tolist()))
#spelling_replacement={'modifier':br_to_us_dict,'head':br_to_us_dict}

def lemma_maker(x, y):
    #print(lemmatizer.lemmatize(x,y))
    return lemmatizer.lemmatize(x,y)
from functools import reduce

parser = argparse.ArgumentParser(description='Compute features from embeddings')

parser.add_argument('--temporal',  type=int,
                    help='Value to bin the temporal information: 0 (remove temporal information), 1 (no binning), 10 (binning to decades), 20 (binning each 20 years) or 50 (binning each 50 years)')

parser.add_argument('--cutoff', type=int, default=50,
                    help='Cut-off frequency for each compound per time period : none (0), 20, 50 and 100')

parser.add_argument('--contextual', action='store_true',
                    help='Is the model contextual')
parser.add_argument('--inputdir',type=str,
                    help='Provide directory where features are located')
parser.add_argument('--outputdir',type=str,
                    help='Where should the output be stored?')

args = parser.parse_args('--contextual --temporal 0 --cutoff 10 --inputdir /home/users0/pageljs/dh/repos/Compounding/coha_compounds/'.split())

In [14]:
print(f'Cutoff: {args.cutoff}')
print(f'Time span:  {args.temporal}')

temp_cutoff_str=str(args.temporal)+'_'+str(args.cutoff)
temp_cutoff_str

Cutoff: 10
Time span:  0


'0_10'

In [15]:
if args.contextual:
    comp_df_path=args.inputdir+'/compounds_CompoundAware_'+temp_cutoff_str+'_300.pkl'
    mod_df_path=args.inputdir+'/modifiers_CompoundAware_'+temp_cutoff_str+'_300.pkl'
    head_df_path=args.inputdir+'/heads_CompoundAware_'+temp_cutoff_str+'_300.pkl'
else:
    comp_df_path=args.inputdir+'/compounds_CompoundAgnostic_'+temp_cutoff_str+'_300.pkl'
    mod_df_path=args.inputdir+'/constituents_CompoundAgnostic_'+temp_cutoff_str+'_300.pkl'
    head_df_path=args.inputdir+'/constituents_CompoundAgnostic_'+temp_cutoff_str+'_300.pkl'

In [16]:
heads=pd.read_pickle(head_df_path)
if args.temporal!=0:
    heads.index.set_names('time', level=1,inplace=True)
    heads.index.set_names('head',level=0,inplace=True)

else:
    heads.index.set_names('head',inplace=True)
heads

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
head,,,,,,,,,,,,,,,,,,,,,
--_noun,0.292,0.208,0.115,-0.016,0.204,0.136,0.069,0.015,0.107,0.002,...,0.002,-0.003,0.008,0.004,0.005,-0.002,-0.004,-0.007,0.002,-0.001
a-runnin_noun,0.016,0.012,0.011,0.004,0.042,0.135,0.209,0.375,0.010,0.002,...,-0.035,-0.009,0.006,-0.043,0.042,-0.020,0.007,-0.029,0.012,0.016
aad_noun,0.039,0.000,0.015,0.007,0.062,0.144,-0.045,-0.017,0.023,0.002,...,-0.006,-0.026,-0.052,-0.033,-0.042,-0.080,-0.001,0.075,-0.012,0.037
abatement_noun,0.019,0.006,0.019,0.010,0.101,0.034,0.071,-0.049,0.017,0.002,...,-0.069,-0.023,-0.001,0.024,-0.005,-0.002,0.001,-0.007,-0.025,-0.030
abbess_noun,0.015,0.005,0.016,0.009,0.019,0.013,0.009,0.001,0.021,0.002,...,0.067,-0.098,-0.053,-0.020,0.103,0.040,-0.038,0.005,0.036,-0.014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zipper_noun,0.009,0.004,0.017,0.011,0.017,0.011,0.006,0.002,0.028,-0.002,...,-0.002,0.029,-0.055,0.048,0.015,0.004,-0.004,0.017,-0.006,-0.029
zis_noun,0.034,0.016,0.018,0.009,0.060,0.029,0.041,0.050,0.030,0.005,...,0.112,-0.015,-0.172,0.018,-0.020,0.081,0.073,0.146,-0.014,-0.045
zone_noun,0.464,-0.089,-0.014,-0.007,-0.023,-0.063,0.028,0.018,-0.005,0.007,...,-0.022,0.024,0.004,0.011,-0.047,-0.024,-0.014,0.028,0.037,-0.016


In [17]:
modifiers=pd.read_pickle(mod_df_path)

if args.temporal!=0:
    modifiers.index.set_names('time', level=1,inplace=True)
    modifiers.index.set_names('modifier',level=0,inplace=True)
else:
    modifiers.index.set_names('modifier',inplace=True)
modifiers

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
modifier,,,,,,,,,,,,,,,,,,,,,
--_noun,0.036,0.005,0.015,0.002,0.055,0.089,-0.020,-0.007,0.032,0.002,...,0.056,0.010,0.074,0.037,0.021,-0.025,-0.030,-0.030,0.037,-0.013
-curity_noun,0.021,-0.001,0.015,0.003,0.051,0.003,0.014,0.000,0.033,0.005,...,-0.026,-0.004,-0.018,0.019,-0.026,-0.033,0.002,-0.011,-0.017,0.014
a-list_noun,0.007,0.004,0.006,0.001,0.029,0.008,0.008,-0.003,0.013,0.002,...,0.101,-0.028,0.124,-0.010,-0.041,0.065,-0.045,-0.012,0.002,-0.039
ab_noun,0.008,0.003,0.006,0.001,0.021,0.009,0.013,-0.000,0.017,0.002,...,0.003,-0.004,-0.039,0.024,-0.006,0.031,0.030,0.016,-0.038,-0.049
abalone_noun,0.007,0.003,0.009,0.004,0.014,0.011,0.010,0.002,0.011,0.001,...,0.081,-0.028,-0.016,0.032,-0.041,0.003,0.005,0.047,0.072,0.140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zombie_noun,0.040,0.021,0.024,0.017,0.060,0.040,0.004,0.003,0.041,0.005,...,-0.006,0.018,0.063,0.030,0.015,-0.023,-0.075,-0.026,-0.001,0.006
zone_noun,0.047,0.004,0.017,0.002,0.093,0.151,0.039,0.020,0.020,0.002,...,0.027,0.003,0.006,0.108,0.011,0.007,0.014,0.085,0.061,0.012
zoning_noun,0.034,0.005,0.034,0.008,0.238,0.001,-0.001,-0.008,0.038,0.006,...,-0.004,0.000,-0.003,-0.021,-0.010,-0.010,-0.021,0.008,-0.020,-0.003


In [18]:
compounds=pd.read_pickle(comp_df_path)

if args.temporal!=0:
    compounds.index.set_names('time', level=2,inplace=True)
compounds.drop(['common'],axis=1,inplace=True)
compounds=compounds+1
compounds

0     1     2     3     4     5     6     7    \
modifier      head                                                             
--_noun       ernment_noun   1.168 0.977 1.024 1.013 1.159 1.811 0.677 0.891   
abortion_noun reform_noun    1.035 1.013 1.015 1.003 1.043 1.018 1.022 0.996   
absentee_noun ballot_noun    1.025 1.008 1.026 1.012 1.062 1.037 1.035 0.997   
              ownership_noun 1.023 1.004 1.009 1.000 1.051 1.006 1.017 0.996   
academy_noun  award_noun     1.074 1.027 1.016 1.003 1.084 1.007 1.022 1.006   
...                            ...   ...   ...   ...   ...   ...   ...   ...   
zebra_noun    mussel_noun    1.032 1.003 1.009 1.001 1.029 1.010 1.019 0.996   
zigzag_noun   course_noun    1.050 1.026 1.016 1.005 1.044 1.023 1.020 1.001   
zinc_noun     chloride_noun  1.023 1.011 1.016 1.015 1.017 1.009 1.014 1.003   
              oxide_noun     1.015 1.012 1.013 1.005 1.045 1.156 1.265 1.468   
zip_noun      code_noun      1.023 1.008 1.022 1.009 1.033 1.018 1.039 0.983   

                               8     9    ...   290   291   292   293   294  \
modifier      head                        ...                                 
--_noun       ernment_noun   0.987 0.990  ... 1.007 1.004 0.993 0.987 1.001   
abortion_noun reform_noun    1.039 1.005  ... 0.986 0.975 1.026 0.990 1.069   
absentee_noun ballot_noun    1.034 1.006  ... 0.948 0.994 0.899 1.023 0.962   
              ownership_noun 1.021 1.002  ... 0.968 0.992 1.123 0.993 0.978   
academy_noun  award_noun     1.035 1.005  ... 0.882 0.989 1.012 1.041 0.979   
...                            ...   ...  ...   ...   ...   ...   ...   ...   
zebra_noun    mussel_noun    1.029 1.003  ... 1.042 0.989 0.998 0.935 1.033   
zigzag_noun   course_noun    1.046 1.005  ... 0.867 1.019 0.980 0.937 0.948   
zinc_noun     chloride_noun  1.037 1.005  ... 0.914 1.122 0.939 1.088 0.958   
              oxide_noun     1.005 1.001  ... 1.015 1.046 1.066 0.958 1.001   
zip_noun      code_noun      1.021 1.003  ... 1.083 0.967 0.966 1.082 0.910   

                               295   296   297   298   299  
modifier      head                                          
--_noun       ernment_noun   1.021 0.980 1.010 0.995 0.999  
abortion_noun reform_noun    1.006 1.020 0.992 0.948 0.994  
absentee_noun ballot_noun    1.032 0.958 0.963 0.997 1.111  
              ownership_noun 0.960 0.926 1.095 1.022 0.905  
academy_noun  award_noun     1.022 0.895 0.939 0.908 1.120  
...                            ...   ...   ...   ...   ...  
zebra_noun    mussel_noun    0.911 0.952 1.063 1.037 0.917  
zigzag_noun   course_noun    1.022 0.892 0.949 0.959 1.065  
zinc_noun     chloride_noun  1.005 1.022 0.850 0.890 1.051  
              oxide_noun     0.976 1.041 0.978 1.022 1.038  
zip_noun      code_noun      0.994 1.039 0.933 1.013 1.119  

[7958 rows x 300 columns]

In [20]:
if args.temporal!=0:
    all_comps=compounds.reset_index()[['modifier','head','time']]
    mod_prod=compounds.groupby(['modifier','time']).size().to_frame()
    mod_prod.columns=['mod_prod']
    head_prod=compounds.groupby(['head','time']).size().to_frame()
    head_prod.columns=['head_prod']
    prod1=pd.merge(all_comps,mod_prod.reset_index(),how='left',on=['modifier','time'])
    productivity=pd.merge(prod1,head_prod.reset_index(),how='left',on=['head','time'])
    productivity.set_index(['modifier','head','time'],inplace=True)
else:
    all_comps=compounds.reset_index()[['modifier','head']]
    mod_prod=compounds.groupby(['modifier']).size().to_frame()
    mod_prod.columns=['mod_prod']
    head_prod=compounds.groupby(['head']).size().to_frame()
    head_prod.columns=['head_prod']
    prod1=pd.merge(all_comps,mod_prod.reset_index(),how='left',on=['modifier'])
    productivity=pd.merge(prod1,head_prod.reset_index(),how='left',on=['head'])
    productivity.set_index(['modifier','head'],inplace=True)
productivity

mod_prod  head_prod
modifier      head                               
--_noun       ernment_noun           1          1
abortion_noun reform_noun            1         22
absentee_noun ballot_noun            2          2
              ownership_noun         2          8
academy_noun  award_noun             1          3
...                                ...        ...
zebra_noun    mussel_noun            1          1
zigzag_noun   course_noun            1         10
zinc_noun     chloride_noun          2          7
              oxide_noun             2          4
zip_noun      code_noun              1          9

[7958 rows x 2 columns]

In [27]:
    all_comps=compounds.reset_index()[['modifier','head']]
    all_comps
    mod_prod=compounds.groupby(['modifier']).size().to_frame()
    mod_prod.columns=['mod_prod']
    head_prod=compounds.groupby(['head']).size().to_frame()
    head_prod.columns=['head_prod']
    prod1=pd.merge(all_comps,mod_prod.reset_index(),how='left',on=['modifier'])
    productivity=pd.merge(prod1,head_prod.reset_index(),how='left',on=['head'])
    productivity.set_index(['modifier','head'],inplace=True)
    productivity

mod_prod  head_prod
modifier      head                               
--_noun       ernment_noun           1          1
abortion_noun reform_noun            1         22
absentee_noun ballot_noun            2          2
              ownership_noun         2          8
academy_noun  award_noun             1          3
...                                ...        ...
zebra_noun    mussel_noun            1          1
zigzag_noun   course_noun            1         10
zinc_noun     chloride_noun          2          7
              oxide_noun             2          4
zip_noun      code_noun              1          9

[7958 rows x 2 columns]